# Pre-Selection Process 2

In [ ]:
import numpy as np
import pandas as pd

# =========================================================
# USER SETTINGS (edit these)
# =========================================================
STORE_COL = "restaurant_id"   # change to your store id column
WEEK_COL = "week_start"       # change to your week column

FEATURE_CATALOG_PATH = "feature_catalog.csv"  # your manually curated file

# Correlation/VIF thresholds (simple defaults)
HIGH_CORR_FLAG = 0.90
MIN_FEATURES_PER_GROUP = 3

VIF_MOD = 5.0
VIF_HIGH = 10.0

# Decision thresholds (simple + explainable)
PAIR_EXPLOSION_THRESHOLD = 10         # if many pairs >= 0.90
VIF_EXPLOSION_FRAC_THRESHOLD = 0.50   # if >=50% features have VIF>=10

# For volatility flagging (per-feature)
VOLATILITY_FLAG_THRESHOLD = 3.0  # median (within-store std / within-store mean_abs_change) heuristic, tune

# =========================================================
# Helpers: parsing
# =========================================================
def parse_bool(x):
    if pd.isna(x): 
        return False
    if isinstance(x, bool): 
        return x
    s = str(x).strip().lower()
    return s in {"true", "1", "yes", "y", "t"}

def parse_list_cell(x):
    """Excel/CSV-friendly: 'mobile,delivery' -> ['mobile','delivery'] ; blank -> []"""
    if pd.isna(x):
        return []
    s = str(x).strip()
    if not s:
        return []
    return [v.strip() for v in s.split(",") if v.strip()]

# =========================================================
# Step A (simplified): Read feature catalog + validate
# =========================================================
def load_feature_catalog(path: str, data: pd.DataFrame) -> pd.DataFrame:
    fc = pd.read_csv(path)

    required = {"feature", "domain", "mechanism_group", "has_lag"}
    missing = required - set(fc.columns)
    if missing:
        raise ValueError(f"feature_catalog.csv missing required columns: {missing}")

    fc = fc.copy()
    fc["has_lag"] = fc["has_lag"].apply(parse_bool)

    if "lag_weeks" not in fc.columns:
        fc["lag_weeks"] = np.nan

    # optional list-like columns
    for col in ["dayparts", "channels", "others"]:
        if col not in fc.columns:
            fc[col] = ""
        fc[col] = fc[col].apply(parse_list_cell)

    # Keep only features that exist in data
    fc["in_data"] = fc["feature"].isin(data.columns)
    bad = fc.loc[~fc["in_data"], "feature"].tolist()
    if bad:
        print(f"[WARN] {len(bad)} features in catalog not found in data. Example: {bad[:10]}")

    fc = fc[fc["in_data"]].reset_index(drop=True)

    # Keep only numeric features for corr/VIF
    numeric_cols = set(data.select_dtypes(include=[np.number]).columns)
    fc["is_numeric"] = fc["feature"].isin(numeric_cols)
    nonnum = fc.loc[~fc["is_numeric"], "feature"].tolist()
    if nonnum:
        print(f"[WARN] {len(nonnum)} features are not numeric; excluded from corr/VIF. Example: {nonnum[:10]}")

    fc = fc[fc["is_numeric"]].reset_index(drop=True)
    return fc

# =========================================================
# Feature-level quality metrics (simple + useful)
# =========================================================
def compute_feature_quality(data: pd.DataFrame, fc: pd.DataFrame) -> pd.DataFrame:
    """
    Outputs per-feature:
      - missing_rate
      - store_availability_rate: % stores having at least one non-null observation
      - volatility_score: median within-store week-to-week abs change / within-store mean abs level (rough)
    """
    features = fc["feature"].tolist()
    out_rows = []

    # Ensure sorted for time deltas
    df = data[[STORE_COL, WEEK_COL] + features].copy()
    df = df.sort_values([STORE_COL, WEEK_COL])

    stores_total = df[STORE_COL].nunique()

    for f in features:
        s = df[f]
        missing_rate = float(s.isna().mean())

        # store availability: store has any non-null value
        store_any = df.groupby(STORE_COL)[f].apply(lambda x: x.notna().any())
        store_availability_rate = float(store_any.mean())  # fraction of stores

        # volatility: within store week-to-week absolute change normalized by typical level
        # (simple & robust; avoids model fitting)
        # 1) compute diff per store
        diff = df.groupby(STORE_COL)[f].diff().abs()
        # 2) typical level (median abs value per store)
        level = df.groupby(STORE_COL)[f].apply(lambda x: np.nanmedian(np.abs(x.values)))
        # 3) typical diff (median abs diff per store)
        diff_med = diff.groupby(df[STORE_COL]).apply(lambda x: np.nanmedian(x.values))

        # avoid divide-by-zero
        vol_per_store = (diff_med / (level.replace(0, np.nan)))
        volatility_score = float(np.nanmedian(vol_per_store.values))

        out_rows.append({
            "feature": f,
            "missing_rate": missing_rate,
            "store_availability_rate": store_availability_rate,
            "volatility_score": volatility_score,
            "volatility_flag": (volatility_score >= VOLATILITY_FLAG_THRESHOLD) if np.isfinite(volatility_score) else False
        })

    return pd.DataFrame(out_rows)

# =========================================================
# Step B: Group correlation matrices + summary
# =========================================================
def corr_summary_for_group(X: pd.DataFrame, high_corr_flag: float) -> dict:
    corr = X.corr(method="pearson")
    # off-diagonal max abs corr
    c = corr.copy()
    np.fill_diagonal(c.values, 0.0)
    abs_c = c.abs()

    iu = np.triu_indices_from(abs_c.values, k=1)
    vals = abs_c.values[iu]
    max_abs = float(np.nanmax(vals)) if len(vals) else np.nan
    n_pairs_flagged = int(np.sum(vals >= high_corr_flag)) if len(vals) else 0

    # keep some top pairs for debugging
    top_pairs = []
    if len(vals):
        top_idx = np.argsort(vals)[::-1][:min(30, len(vals))]
        cols = abs_c.columns.to_list()
        for idx in top_idx:
            i, j = iu[0][idx], iu[1][idx]
            top_pairs.append((cols[i], cols[j], float(vals[idx])))
    top_pairs_df = pd.DataFrame(top_pairs, columns=["feature_a", "feature_b", "abs_corr"])

    return {
        "corr_pearson": corr,
        "max_abs_corr_offdiag": max_abs,
        "n_pairs_flagged_ge_0p90": n_pairs_flagged,
        "top_abs_corr_pairs": top_pairs_df,
    }

# =========================================================
# Step D: VIF within group (simple, stable)
# =========================================================
def zscore_df(X: pd.DataFrame) -> pd.DataFrame:
    mu = X.mean(axis=0, skipna=True)
    sd = X.std(axis=0, ddof=0, skipna=True).replace(0, np.nan)
    return (X - mu) / sd

def vif_from_corr(C: np.ndarray, names: list, ridge_eps=1e-8) -> pd.Series:
    C = np.nan_to_num(C, nan=0.0, posinf=0.0, neginf=0.0)
    C = (C + C.T) / 2.0
    np.fill_diagonal(C, 1.0)
    C = C + np.eye(C.shape[0]) * ridge_eps
    try:
        invC = np.linalg.inv(C)
    except np.linalg.LinAlgError:
        invC = np.linalg.pinv(C)
    return pd.Series(np.diag(invC), index=names)

def vif_summary_for_group(X: pd.DataFrame) -> dict:
    # Drop near-constant columns
    nunique = X.nunique(dropna=True)
    var = X.var(numeric_only=True)
    keep_cols = X.columns[(nunique > 2) & (var > 1e-12)]
    X = X[keep_cols].copy()

    if X.shape[1] < 2:
        return {"vif_table": pd.DataFrame(), "vif_max": np.nan, "vif_p95": np.nan, "vif_median": np.nan,
                "n_vif_ge_10": 0, "n_vif_ge_5": 0, "top_vif_feature": None, "top_vif_value": np.nan}

    # For VIF we need a reasonable number of rows; use rows with not-too-much missingness
    row_missing = X.isna().mean(axis=1)
    X = X.loc[row_missing <= 0.5]
    if len(X) < 100:
        return {"vif_table": pd.DataFrame(), "vif_max": np.nan, "vif_p95": np.nan, "vif_median": np.nan,
                "n_vif_ge_10": 0, "n_vif_ge_5": 0, "top_vif_feature": None, "top_vif_value": np.nan}

    Z = zscore_df(X)
    Z = Z.loc[:, Z.notna().any(axis=0)]
    if Z.shape[1] < 2:
        return {"vif_table": pd.DataFrame(), "vif_max": np.nan, "vif_p95": np.nan, "vif_median": np.nan,
                "n_vif_ge_10": 0, "n_vif_ge_5": 0, "top_vif_feature": None, "top_vif_value": np.nan}

    C = Z.corr(method="pearson").to_numpy()
    vif = vif_from_corr(C, Z.columns.to_list())

    vif_tbl = pd.DataFrame({"feature": vif.index, "vif": vif.values}).sort_values("vif", ascending=False).reset_index(drop=True)

    vif_max = float(vif_tbl["vif"].max())
    vif_p95 = float(vif_tbl["vif"].quantile(0.95))
    vif_median = float(vif_tbl["vif"].median())
    n_ge_10 = int((vif_tbl["vif"] >= VIF_HIGH).sum())
    n_ge_5 = int((vif_tbl["vif"] >= VIF_MOD).sum())
    top_feat = str(vif_tbl.iloc[0]["feature"])
    top_val = float(vif_tbl.iloc[0]["vif"])

    return {
        "vif_table": vif_tbl,
        "vif_max": vif_max,
        "vif_p95": vif_p95,
        "vif_median": vif_median,
        "n_vif_ge_10": n_ge_10,
        "n_vif_ge_5": n_ge_5,
        "top_vif_feature": top_feat,
        "top_vif_value": top_val,
    }

# =========================================================
# Step E (simplified): decision logic + optional collapse
# =========================================================
def decide_group(n_features: int,
                 max_abs_corr_offdiag: float,
                 n_pairs_flagged_ge_0p90: int,
                 vif_max: float,
                 n_vif_ge_10: int) -> tuple[str, str]:
    """
    Returns (decision, rationale).
    Decisions:
      - KEEP: keep as-is
      - KEEP_REPS: keep 1 rep per high-corr cluster (simple)
      - COLLAPSE_TO_INDEX: create one index feature (simple mean)
    """
    # compute fraction high VIF
    frac_vif_ge_10 = (n_vif_ge_10 / n_features) if n_features else 0.0

    # Explosion triggers
    if (n_pairs_flagged_ge_0p90 >= PAIR_EXPLOSION_THRESHOLD) or (frac_vif_ge_10 >= VIF_EXPLOSION_FRAC_THRESHOLD):
        return ("COLLAPSE_TO_INDEX",
                f"High redundancy: pairs>=0.90={n_pairs_flagged_ge_0p90}, frac(VIF>=10)={frac_vif_ge_10:.2f}. Collapse to index.")

    # Moderate redundancy triggers
    if (max_abs_corr_offdiag is not None and max_abs_corr_offdiag >= HIGH_CORR_FLAG) or (vif_max is not None and vif_max >= VIF_HIGH):
        return ("KEEP_REPS",
                f"Moderate/high redundancy signal: max_abs_corr={max_abs_corr_offdiag:.3f}, vif_max={vif_max:.1f}. Keep representatives per cluster.")

    return ("KEEP",
            f"Low redundancy: max_abs_corr={max_abs_corr_offdiag:.3f}, vif_max={vif_max:.1f}. Keep group features.")

def choose_rep_simple(features: list[str], feature_quality: pd.DataFrame) -> str:
    """
    Simple representative choice:
      - lowest missing_rate
      - highest store_availability
      - lowest volatility_score
    """
    q = feature_quality.set_index("feature").loc[features].copy()
    q["missing_rate"] = q["missing_rate"].fillna(1.0)
    q["store_availability_rate"] = q["store_availability_rate"].fillna(0.0)
    q["volatility_score"] = q["volatility_score"].fillna(np.inf)

    q = q.sort_values(
        ["missing_rate", "store_availability_rate", "volatility_score"],
        ascending=[True, False, True]
    )
    return q.index[0]

# Very simple clustering by correlation adjacency (connected components)
def corr_clusters(corr_abs: pd.DataFrame, threshold: float) -> list[list[str]]:
    cols = corr_abs.columns.tolist()
    adj = {c: set() for c in cols}
    M = corr_abs.values
    n = len(cols)
    for i in range(n):
        for j in range(i+1, n):
            if M[i, j] >= threshold:
                a, b = cols[i], cols[j]
                adj[a].add(b); adj[b].add(a)
    seen = set()
    comps = []
    for c in cols:
        if c in seen:
            continue
        stack = [c]
        seen.add(c)
        comp = []
        while stack:
            u = stack.pop()
            comp.append(u)
            for v in adj[u]:
                if v not in seen:
                    seen.add(v)
                    stack.append(v)
        comps.append(sorted(comp))
    return comps

# =========================================================
# MAIN: run simplified pipeline B + D + E
# =========================================================
def run_simple_pipeline(data: pd.DataFrame, feature_catalog_csv: str):
    # Load catalog
    fc = load_feature_catalog(feature_catalog_csv, data)

    # Feature quality
    feature_quality = compute_feature_quality(data, fc)

    # Build groups
    group_keys = ["domain", "mechanism_group", "has_lag", "lag_weeks"]
    grouped = fc.groupby(group_keys, dropna=False)

    covariate_matrices = {}  # group_key -> {corr_pearson, top pairs, ...}
    vif_details = {}         # group_key -> vif_table
    summary_rows = []
    mapping_rows = []
    collapse_definitions = {}

    # Iterate each group
    for gkey, gdf in grouped:
        features = gdf["feature"].tolist()
        if len(features) < MIN_FEATURES_PER_GROUP:
            continue

        X = data[features]

        # Step B: correlation
        corr_out = corr_summary_for_group(X, HIGH_CORR_FLAG)
        covariate_matrices[gkey] = corr_out

        # Step D: VIF
        vif_out = vif_summary_for_group(X)
        vif_details[gkey] = vif_out["vif_table"]

        # Step E: decision
        decision, rationale = decide_group(
            n_features=len(features),
            max_abs_corr_offdiag=corr_out["max_abs_corr_offdiag"],
            n_pairs_flagged_ge_0p90=corr_out["n_pairs_flagged_ge_0p90"],
            vif_max=vif_out["vif_max"],
            n_vif_ge_10=vif_out["n_vif_ge_10"]
        )

        # Quality summaries at group level
        fq = feature_quality.set_index("feature").loc[features]
        miss_mean = float(fq["missing_rate"].mean())
        avail_mean = float(fq["store_availability_rate"].mean())
        vol_med = float(np.nanmedian(fq["volatility_score"].values))

        # Apply the decision (simple actions)
        selected = []
        if decision == "KEEP":
            selected = features
            for f in features:
                mapping_rows.append({"group_key": gkey, "feature": f, "action": "KEEP", "mapped_to": f})

        elif decision == "KEEP_REPS":
            # cluster by corr >= 0.90 and keep one rep per cluster
            corr_abs = corr_out["corr_pearson"].abs()
            clusters = corr_clusters(corr_abs, HIGH_CORR_FLAG)
            for ci, cl in enumerate(clusters):
                rep = choose_rep_simple(cl, feature_quality)
                selected.append(rep)
                for f in cl:
                    mapping_rows.append({"group_key": gkey, "feature": f, "action": "REP" if f == rep else "MAP_TO_REP", "mapped_to": rep})

        elif decision == "COLLAPSE_TO_INDEX":
            # create one simple mean index (exec-friendly)
            index_name = f"{gkey[0]}__{gkey[1]}__{'lag'+str(int(gkey[3])) if parse_bool(gkey[2]) else 'nolag'}__idx"
            data[index_name] = data[features].mean(axis=1, skipna=True)

            selected = [index_name]
            collapse_definitions[index_name] = {
                "type": "mean_index",
                "group_key": str(gkey),
                "inputs": features
            }
            for f in features:
                mapping_rows.append({"group_key": gkey, "feature": f, "action": "COLLAPSE_TO_INDEX", "mapped_to": index_name})

        # Build summary row (what you asked for)
        summary_rows.append({
            "group_key": gkey,
            "domain": gkey[0],
            "mechanism_group": gkey[1],
            "has_lag": gkey[2],
            "lag_weeks": gkey[3],
            "n_features": len(features),

            # feature quality summary
            "missing_rate_mean": miss_mean,
            "store_availability_rate_mean": avail_mean,
            "volatility_score_median": vol_med,

            # corr summary
            "max_abs_corr_offdiag": corr_out["max_abs_corr_offdiag"],
            "n_pairs_flagged_ge_0p90": corr_out["n_pairs_flagged_ge_0p90"],

            # vif summary
            "vif_max": vif_out["vif_max"],
            "vif_p95": vif_out["vif_p95"],
            "vif_median": vif_out["vif_median"],
            "n_vif_ge_10": vif_out["n_vif_ge_10"],
            "n_vif_ge_5": vif_out["n_vif_ge_5"],
            "top_vif_feature": vif_out["top_vif_feature"],
            "top_vif_value": vif_out["top_vif_value"],

            # decision
            "decision": decision,
            "rationale": rationale,

            # what got selected
            "n_selected_from_group": len(selected),
            "selected_preview": ", ".join(selected[:5]) + ("..." if len(selected) > 5 else "")
        })

    summary = pd.DataFrame(summary_rows).sort_values(
        ["decision", "n_features", "n_pairs_flagged_ge_0p90", "vif_max"],
        ascending=[True, False, False, False]
    ).reset_index(drop=True)

    mapping_table = pd.DataFrame(mapping_rows)

    # Final selected set:
    final_selected = set(mapping_table.loc[mapping_table["action"].isin(["KEEP", "REP"]), "mapped_to"].tolist())
    final_selected |= set(mapping_table.loc[mapping_table["action"] == "COLLAPSE_TO_INDEX", "mapped_to"].tolist())

    # add IDs
    for col in [STORE_COL, WEEK_COL]:
        if col in data.columns:
            final_selected.add(col)

    final_selected = sorted(final_selected)

    return {
        "summary": summary,
        "mapping_table": mapping_table,
        "collapse_definitions": collapse_definitions,
        "final_selected_features": final_selected,
        "feature_quality": feature_quality,
        "covariate_matrices": covariate_matrices,  # for detailed lookups
        "vif_details": vif_details,                # per-group VIF tables
    }

# =========================================================
# RUN
# =========================================================
# Example usage:
# data = pd.read_parquet("your_store_week_data.parquet")  # or however you load
results = run_simple_pipeline(data, FEATURE_CATALOG_PATH)

summary = results["summary"]
mapping_table = results["mapping_table"]
final_features = results["final_selected_features"]

print("Final selected features:", len(final_features))
print(summary.head(20))

# =========================================================
# HOW TO PULL DETAILS (if someone asks)
# =========================================================
# Pick a group_key from summary:
# gkey = summary.loc[0, "group_key"]
# corr_matrix = results["covariate_matrices"][gkey]["corr_pearson"]
# top_pairs = results["covariate_matrices"][gkey]["top_abs_corr_pairs"]
# vif_table = results["vif_details"][gkey]
